In [1]:
import numpy as np
import time
import stock
import kline

In [5]:
c,k,d = stock.loadKline('SZ002230')
bo = stock.bollLineK(k,20)
vbo = stock.boll(k[:,0],20)
kline.Plote('SZ002230','d',config={'figure':[[{
    'data':stock.bollWidth(bo)
}],[{
    'data':stock.bollWidth(vbo),
    'color':'red'
}]]}).show()

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()

In [3]:
kline.Plote('SH600519').show()

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()

In [4]:
kline.Plote('SH:601099').show()

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()

In [3]:
c,kline,date = stock.loadKline('SH:601099',15)